In [1]:
#install all necessary libraries
from bs4 import BeautifulSoup
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
import pandas as pd

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 3, Finished, Available)

In [2]:
# Initialize SparkSession
spark = SparkSession.builder.getOrCreate()

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 4, Finished, Available)

In [3]:
#creating a function to scrape the data

def scrape_table(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    table = soup.find('table')

    headers = []
    for th in table.find_all('th'):
        headers.append(th.text.strip())

    rows = []
    for row in table.find_all('tr'):
        cells = row.find_all('td')
        if cells:
            rows.append([cell.text.strip() for cell in cells])

    return headers, rows

url = "https://www.worldometers.info/world-population/population-by-country/"
headers, rows = scrape_table(url)

spark = SparkSession.builder.getOrCreate()

schema = StructType([StructField(header, StringType(), True) for header in headers])

population = spark.createDataFrame(rows, schema)

display(population.limit(10))

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, b981cd37-601a-4c51-89c8-5f5f83ecd081)

In [4]:
# Rename the column "Country (or dependency)" to "Country"
population = population.withColumnRenamed("Country (or dependency)", "Country")
display(population.limit(5))

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 20014beb-54c2-4534-9011-e6c4f56cbd39)

In [5]:
# Rename the column
population = population.withColumnRenamed("#", "S/N")
display(population.limit(5))

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 6264927b-cdb9-44d1-be9e-5f96f9b4e601)

In [6]:
# Make a request to the website to scrape the GDP of each country
# The DataFrame used is that of Pandas
url = 'https://www.worldometers.info/gdp/gdp-by-country/'
response = requests.get(url)

# Parse the whole HTML page using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table on the webpage
table = soup.find_all('table')[0] 

# Read the table into a pandas DataFrame
gdp = pd.read_html(str(table), header=0)[0]

# Print the DataFrame #print(df)
gdp.head(5)

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 8, Finished, Available)

,#,Country,"GDP (nominal, 2022)",GDP (abbrev.),GDP growth,Population (2022),GDP per capita,Share of World GDP
0,1,United States,"$25,462,700,000,000",$25.463 trillion,2.06%,338289857,"$75,269",25.32%
1,2,China,"$17,963,200,000,000",$17.963 trillion,2.99%,1425887337,"$12,598",17.86%
2,3,Japan,"$4,231,140,000,000",$4.231 trillion,1.03%,123951692,"$34,135",4.21%
3,4,Germany,"$4,072,190,000,000",$4.072 trillion,1.79%,83369843,"$48,845",4.05%
4,5,India,"$3,385,090,000,000",$3.385 trillion,7.00%,1417173173,"$2,389",3.37%


In [7]:
column_list = gdp.columns.tolist()
print(column_list)

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 9, Finished, Available)

['#', 'Country', 'GDP  (nominal, 2022)', 'GDP  (abbrev.)', 'GDP growth', 'Population  (2022)', 'GDP per capita', 'Share of  World GDP']


In [8]:
#view the first 5 rows of this column
gdp['GDP  (nominal, 2022)'].head(5)

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 10, Finished, Available)

0    $25,462,700,000,000
1    $17,963,200,000,000
2     $4,231,140,000,000
3     $4,072,190,000,000
4     $3,385,090,000,000
Name: GDP  (nominal, 2022), dtype: object

In [9]:
# Remove dollar sign and comma separator from 'GDP (nominal, 2022)'

gdp['GDP  (nominal, 2022)'] = gdp['GDP  (nominal, 2022)'].replace({'\$': '', ',': ''}, regex=True)
gdp.head(5)

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 11, Finished, Available)

,#,Country,"GDP (nominal, 2022)",GDP (abbrev.),GDP growth,Population (2022),GDP per capita,Share of World GDP
0,1,United States,25462700000000,$25.463 trillion,2.06%,338289857,"$75,269",25.32%
1,2,China,17963200000000,$17.963 trillion,2.99%,1425887337,"$12,598",17.86%
2,3,Japan,4231140000000,$4.231 trillion,1.03%,123951692,"$34,135",4.21%
3,4,Germany,4072190000000,$4.072 trillion,1.79%,83369843,"$48,845",4.05%
4,5,India,3385090000000,$3.385 trillion,7.00%,1417173173,"$2,389",3.37%


In [10]:
#drop other columns but retain on the first 3 columns
gdp = gdp.iloc[:, :3]
gdp.head(5)

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 12, Finished, Available)

,#,Country,"GDP (nominal, 2022)"
0,1,United States,25462700000000
1,2,China,17963200000000
2,3,Japan,4231140000000
3,4,Germany,4072190000000
4,5,India,3385090000000


In [12]:
# Scrape the Capital and Currency of each country
url = 'https://catking.in/list-of-countries-capitals-currencies-of-the-world/'
response = requests.get(url)

# Parse the whole HTML page using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table on the webpage
table = soup.find_all('table')[0] 

# Read the table into a pandas DataFrame
df = pd.read_html(str(table), header=0)[0]

# Convert the pandas DataFrame to PySpark DataFrame
capital = spark.createDataFrame(df)

# Show the DataFrame
display(capital.limit(5))

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 14, Finished, Available)

SynapseWidget(Synapse.DataFrame, 17144cbf-34d6-4faa-9777-b164b1eadea6)

In [13]:
# Rename the column "Country Name" to "Country"
capital = capital.withColumnRenamed("Country Name", "Country")
display(capital.limit(5))

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 15, Finished, Available)

SynapseWidget(Synapse.DataFrame, a0dec1ed-f4c2-4eef-8fc8-5f71b6c0027e)

In [17]:
#replace 'United States of America' with 'United States' in the 'Country' column
from pyspark.sql.functions import col, when

spark = SparkSession.builder.appName("ReplaceRecord").getOrCreate()

capital = capital.withColumn("Country", when(col("Country") == "United States of America", "United States").otherwise(col("Country")))

display(capital.limit(5))

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 19, Finished, Available)

SynapseWidget(Synapse.DataFrame, b1384647-69c9-47ba-8655-e5b0195251fd)

In [18]:
# Scrape the name of Presidents and Prime Ministers 
url = "https://www.geeksforgeeks.org/list-of-presidents-prime-ministers-around-the-world/"

# Send HTTP request to the specified URL and save the response from server in a response object called r
r = requests.get(url)

# Create a BeautifulSoup object and specify the parser
soup = BeautifulSoup(r.text, 'html.parser')

# Find the table in the webpage
table = soup.find_all('table')[0] 

# Use pandas to read the table into a pandas DataFrame
df = pd.read_html(str(table), header=0)[0]

# Convert pandas DataFrame to PySpark DataFrame
president = spark.createDataFrame(df)

# Show the DataFrame
display(president.limit(5))

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 20, Finished, Available)

SynapseWidget(Synapse.DataFrame, 676be121-34d0-4d16-9b56-bd34e7ac3bcc)

In [19]:
# Perform the join operation on few tables fo testing
joined_df = capital.join(president, capital["Country"] == president["Country"], 'left')

# Show the result
display(joined_df.limit(5))

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 21, Finished, Available)

SynapseWidget(Synapse.DataFrame, e41748e4-50ba-4f43-9f58-00316a4a04aa)

In [20]:
#Convert the gdp pandas dataframe into spark dataframe
# Create a SparkSession
spark_gdp = SparkSession.builder.appName("MyApp").getOrCreate()

# Convert the Pandas DataFrame to a PySpark DataFrame
spark_gdp = spark.createDataFrame(gdp)

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 22, Finished, Available)

In [21]:
#test the spark_gdp dataframe on population pyspark dataframe
result2 = population.join(spark_gdp, on="Country", how="left") 
                
display(result2.limit(4))

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 23, Finished, Available)

SynapseWidget(Synapse.DataFrame, bc10fe57-e569-4465-a11b-cbb993547e56)

In [22]:
#Final join
result = capital.join(population, on="Country", how="left") \
                .join(president, on="Country", how="left") \
                .join(spark_gdp, on="Country", how="left") 

display(result.limit(5))

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 24, Finished, Available)

SynapseWidget(Synapse.DataFrame, f08c8f6f-0a20-47b0-8777-753182faa0c9)

In [23]:
# Remove dollar sign and comma separator from 'GDP (nominal, 2022)'
from pyspark.sql.functions import col, regexp_replace


result = result.withColumn('Population (2023)', regexp_replace(col('Population (2023)'), '[$,]', ''))
result = result.withColumn('Land Area (Km²)', regexp_replace(col('Land Area (Km²)'), '[$,]', ''))
display(result.limit(5))

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 25, Finished, Available)

SynapseWidget(Synapse.DataFrame, 08782dca-1529-4fc6-ae6b-3f22a1aa2d98)

In [24]:
#drop unnecessary columns
result = result.drop('Sr. No.', 'S/N', 'Yearly Change','Density (P/Km²)','Migrants (net)','#')
display(result.limit(3))

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 26, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9332f48f-9e6a-4bf8-a181-e8156c9b319d)

In [25]:
#rename columns that could throw out error
result = result.withColumnRenamed('Population (2023)', 'Population2023')\
       .withColumnRenamed('Land Area (Km²)', 'LandAreaKm2')\
       .withColumnRenamed('Fert. Rate', 'FertilityRate')\
       .withColumnRenamed('Med. Age', 'MedicalAge')\
       .withColumnRenamed('Urban Pop %', 'UrbanPopulation')\
       .withColumnRenamed('Med. Age', 'MedicalAge')\
       .withColumnRenamed('Urban Pop %', 'UrbanPopulation')\
       .withColumnRenamed('Med. Age', 'MedicalAge')\
       .withColumnRenamed('President/Head of Country', 'President')\
       .withColumnRenamed('GDP  (nominal, 2022)', 'GDP2022')\
       .withColumnRenamed('Net Change', 'NetChange')\
       .withColumnRenamed('World Share', 'WorldShare')\
       .withColumnRenamed('Urban Pop %', 'UrbanPopulation')\
       .withColumnRenamed('Prime Minister', 'PrimeMinister')
       
display(result.limit(2))

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 27, Finished, Available)

SynapseWidget(Synapse.DataFrame, 033ab864-819f-4523-a89d-65c4b5c92458)

In [28]:
# Change datatype of some columns 

spark = SparkSession.builder.appName("ChangeDataType").getOrCreate()

result = result.withColumn("Population2023", col("Population2023").cast("integer"))
result = result.withColumn("LandAreaKm2", col("LandAreaKm2").cast("integer"))
result = result.withColumn("MedicalAge", col("MedicalAge").cast("integer"))
result = result.withColumn("GDP2022", col("GDP2022").cast("integer"))
result = result.withColumn("FertilityRate", col("FertilityRate").cast("float"))

display(result.limit(5))

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 30, Finished, Available)

SynapseWidget(Synapse.DataFrame, 94cabcf7-1ed5-460b-b981-c2b94a78bb48)

In [32]:
#write dataframe to LakeHouse Table

result.write.format("delta").mode("overwrite").save("Tables/WorldData")

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 34, Finished, Available)

In [33]:
#verify content in the table

df = spark.sql("SELECT * FROM World_DataProject.WorldData LIMIT 5")
display(df)

StatementMeta(, 830817f2-6d65-4e0a-bd0a-ddd078b0377d, 35, Finished, Available)

SynapseWidget(Synapse.DataFrame, daa824fc-2d46-4b1d-a7ae-a48befa37f48)